### this code is from "Worth Web Scrapping" YouTube channel 
See tutorial for 
[Scraping Amazon Reviews using Python](https://www.youtube.com/watch?v=5secVgADRpA)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
search_query = "t-shirt"
base_url = "https://www.amazon.com/s?k="

In [ ]:
url = base_url+search_query
url

In [ ]:
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36','referer':'https://www.amazon.com/s?k=t-shirt&crid=SI7RFOQTLIZ4&sprefix=t-shir%2Caps%2C294&ref=nb_sb_noss'}

In [ ]:
search_response=requests.get(url,headers=header)

In [ ]:
search_response.status_code

In [ ]:
search_response.text

In [ ]:
search_response.cookies

 ## function to get the content of the page of required query

In [ ]:
cookie={} # insert request cookies within{}
def getAmazonSearch(search_query):
    url="https://www.amazon.com/s?k="+search_query
    print(url)
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [ ]:
getAmazonSearch('pillow')

## function to get the contents of individual product pages using 'data-asin' number (unique identification number)

In [ ]:
def Searchasin(asin):
    url="https://www.amazon.com/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [ ]:
Searchasin('B07YDYPVGF')

## function to pass on the link of 'see all reviews' and extract the content


In [ ]:
def Searchreviews(review_link):
    url="https://www.amazon.com"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

## First page product reviews extraction

In [ ]:
product_names=[]
response=getAmazonSearch(search_query)
soup=BeautifulSoup(response.content)
for i in soup.findAll("span",{'class':'a-size-base-plus a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list


In [ ]:
product_names

In [ ]:
len(product_names)

#### The method of extracting data-asin numbers are similar to that of product names. Only the tag details have to be changed in findall()


In [ ]:
data_asin=[]
response=getAmazonSearch(search_query)
soup=BeautifulSoup(response.content)
for i in soup.findAll("div",{'class':"sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20"}):
    data_asin.append(i['data-asin'])


In [ ]:
response.status_code

In [ ]:
data_asin

In [ ]:
len(data_asin)

#### By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [ ]:
link=[]
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

In [ ]:
link


#### Now we have the 'see all review' links. Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [ ]:
reviews=[]
for j in range(len(link)):
    for k in range(100):
        response=Searchreviews(link[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text)

In [ ]:
reviews

In [ ]:
len(reviews)

In [ ]:
rev={'reviews':reviews}

In [ ]:
review_data=pd.DataFrame.from_dict(rev)


In [ ]:
review_data.head(5)

In [ ]:
review_data.to_csv('tshirt_reviews.csv', index = False) #converting the dataframe to a csv file so as to use it later for further analysis